In [1]:
import pandas as pd

df_features = pd.read_json('/kaggle/input/new-codalab-final-test/training_data/train.features',lines=True)
df_labels = pd.read_json('/kaggle/input/new-codalab-final-test/training_data/train.labels',lines=True)

In [2]:
df = pd.merge(df_features,df_labels,on="indoml_id")

In [3]:
df

,indoml_id,description,retailer,price,supergroup,group,module,brand
0,0,1 adblue,organicorner,25.35,automotive,automotive detail unknown total,automotive,receipt all
1,1,1 car mat set,greenharbor,4.99,automotive,automotive detail unknown total,automotive,receipt all
2,2,1 cp rmx scrnwash,naturify,3.85,automotive,automotive detail unknown total,automotive,receipt all
3,3,1 diesel,ecogro,4.41,automotive,automotive detail unknown total,automotive,receipt all
4,4,1 unstoppable refrsher,greenharbor,3.00,automotive,automotive detail unknown total,automotive,receipt all
...,...,...,...,...,...,...,...,...
561833,561833,zuru xshot excelxcess,noshify,16.99,toys,toys detail unknown total,toys,receipt all
561834,561834,zuru xshot micro,vitalveg,3.50,toys,toys detail unknown total,toys,receipt all
561835,561835,zuru xshot typhoon thunder,crispcorner,8.50,toys,toys detail unknown total,toys,receipt all
561836,561836,zzand,snackify,4.79,toys,toys detail unknown total,toys,receipt all


In [4]:
len(df["brand"].unique())

5679

In [5]:
from sklearn.preprocessing import LabelEncoder
group_encoder = LabelEncoder()
supergroup_encoder = LabelEncoder()
module_encoder = LabelEncoder()
brand_encoder = LabelEncoder()

# Fit and transform each column
df['group'] = group_encoder.fit_transform(df['group'])
df['supergroup'] = supergroup_encoder.fit_transform(df['supergroup'])
df['module'] = module_encoder.fit_transform(df['module'])
df['brand'] = brand_encoder.fit_transform(df['brand'])

df['description'] = df['description'] + " " + df['retailer'].astype(str) +" " +df["price"].astype(str)

In [6]:
brand_encoder.classes_

array(['& soda', '10 motives', '1001', ..., 'zumi', 'zylkene', 'zyn'],
      dtype=object)

In [7]:
df

,indoml_id,description,retailer,price,supergroup,group,module,brand
0,0,1 adblue organicorner 25.35,organicorner,25.35,0,2,18,4078
1,1,1 car mat set greenharbor 4.99,greenharbor,4.99,0,2,18,4078
2,2,1 cp rmx scrnwash naturify 3.85,naturify,3.85,0,2,18,4078
3,3,1 diesel ecogro 4.41,ecogro,4.41,0,2,18,4078
4,4,1 unstoppable refrsher greenharbor 3.0,greenharbor,3.00,0,2,18,4078
...,...,...,...,...,...,...,...,...
561833,561833,zuru xshot excelxcess noshify 16.99,noshify,16.99,31,215,407,4078
561834,561834,zuru xshot micro vitalveg 3.5,vitalveg,3.50,31,215,407,4078
561835,561835,zuru xshot typhoon thunder crispcorner 8.5,crispcorner,8.50,31,215,407,4078
561836,561836,zzand snackify 4.79,snackify,4.79,31,215,407,4078


In [8]:
supergroup_dict = {}
for val in df["supergroup"].unique():
    new_df = df[df["supergroup"]==val]
    supergroup_dict[val] = new_df["group"].unique().tolist()

In [9]:
group_dict = {}
for val in df["group"].unique():
    new_df = df[df["group"]==val]
    group_dict[val] = new_df["module"].unique().tolist()

In [10]:
module_dict = {}
for val in df["module"].unique():
    new_df = df[df["module"]==val]
    module_dict[val] = new_df["brand"].unique().tolist()

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
# from transformers import DistilBertTokenizer, DistilBertModel
from transformers import BertForSequenceClassification
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.amp import autocast, GradScaler

# Constants
MAX_LENGTH = 20
BATCH_SIZE = 32
LEARNING_RATE = 5e-5
NUM_EPOCHS = 0
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
PATIENCE = 5  # Early stopping patience
PATIENCE_LR = 3  # Reduce LR on plateau patience

In [12]:
!pip install torch_optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 3.6 MB/s eta 0:00:00


In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler
from torch_optimizer import AdamP

# Constants (keep the existing constants like MAX_LENGTH, BATCH_SIZE, etc.)

tokenizer = AutoTokenizer.from_pretrained("canwenxu/BERT-of-Theseus-MNLI")


def custom_hierarchical_loss(logits, labels, valid_indices):
    batch_size = logits.size(0)
    loss = 0
    for i in range(batch_size):
        valid_idx = torch.tensor(list(valid_indices[i]), device=logits.device)
        if valid_idx.numel() == 0:
            continue
        valid_logits = logits[i, valid_idx]
        label = labels[i]
        if label not in valid_idx:
            label = valid_idx[0]
        label_idx = torch.where(valid_idx == label)[0]
        if label_idx.numel() == 0:
            continue
        loss += nn.CrossEntropyLoss()(valid_logits.unsqueeze(0), label_idx)
    return loss / batch_size if batch_size > 0 else loss

# Separate Model for each level of hierarchy
class SupergroupModel(nn.Module):
    def __init__(self, num_supergroups):
        super(SupergroupModel, self).__init__()
        self.base_model = AutoModel.from_pretrained("canwenxu/BERT-of-Theseus-MNLI")
        self.supergroup_classifier = nn.Linear(self.base_model.config.hidden_size, num_supergroups)

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0, :]
        return self.supergroup_classifier(pooled_output)

class GroupModel(nn.Module):
    def __init__(self, num_groups):
        super(GroupModel, self).__init__()
        self.base_model = AutoModel.from_pretrained("canwenxu/BERT-of-Theseus-MNLI")
        self.group_classifier = nn.Linear(self.base_model.config.hidden_size, num_groups)
        self.supergroup_to_group = supergroup_dict  # Hierarchical mapping

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0, :]
        return self.group_classifier(pooled_output)

    def predict(self, input_ids, attention_mask, supergroup_preds):
        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        group_probs = torch.softmax(logits, dim=1)
        
        group_preds = []
        batch_size = supergroup_preds.size(0)

        for i in range(batch_size):
            supergroup = supergroup_preds[i].item()
            valid_groups = self.supergroup_to_group.get(supergroup, range(logits.size(1)))
            valid_group_probs = group_probs[i, valid_groups]
            group_pred = valid_groups[torch.argmax(valid_group_probs).item()]
            group_preds.append(group_pred)

        return torch.tensor(group_preds).to(DEVICE)

class ModuleModel(nn.Module):
    def __init__(self, num_modules):
        super(ModuleModel, self).__init__()
        self.base_model = AutoModel.from_pretrained("canwenxu/BERT-of-Theseus-MNLI")
        self.module_classifier = nn.Linear(self.base_model.config.hidden_size, num_modules)
        self.group_to_module = group_dict  # Hierarchical mapping

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0, :]
        return self.module_classifier(pooled_output)

    def predict(self, input_ids, attention_mask, group_preds):
        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        module_probs = torch.softmax(logits, dim=1)
        
        module_preds = []
        batch_size = group_preds.size(0)

        for i in range(batch_size):
            group = group_preds[i].item()
            valid_modules = self.group_to_module.get(group, range(logits.size(1)))
            valid_module_probs = module_probs[i, valid_modules]
            module_pred = valid_modules[torch.argmax(valid_module_probs).item()]
            module_preds.append(module_pred)

        return torch.tensor(module_preds).to(DEVICE)

class BrandModel(nn.Module):
    def __init__(self, num_brands):
        super(BrandModel, self).__init__()
        self.base_model = AutoModel.from_pretrained("canwenxu/BERT-of-Theseus-MNLI")
        self.brand_classifier = nn.Linear(self.base_model.config.hidden_size, num_brands)
        self.module_to_brand = module_dict  # Hierarchical mapping from module to brand

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0, :]
        return self.brand_classifier(pooled_output)

    def predict(self, input_ids, attention_mask, module_preds):
        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        brand_probs = torch.softmax(logits, dim=1)

        brand_preds = []
        batch_size = module_preds.size(0)

        for i in range(batch_size):
            module = module_preds[i].item()
            valid_brands = self.module_to_brand.get(module, range(logits.size(1)))
            valid_brand_probs = brand_probs[i, valid_brands]
            brand_pred = valid_brands[torch.argmax(valid_brand_probs).item()]
            brand_preds.append(brand_pred)

        return torch.tensor(brand_preds).to(DEVICE)


# Dataset class remains the same
class ProductDataset(Dataset):
    def __init__(self, texts, labels1, labels2, labels3, labels4):
        self.texts = texts
        self.labels1 = labels1  # Supergroup
        self.labels2 = labels2  # Group
        self.labels3 = labels3  # Module
        self.labels4 = labels4  # Brand
        self.encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors="pt")

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        item = {key: val[idx].to(DEVICE) for key, val in self.encodings.items()}
        item['labels1'] = torch.tensor(self.labels1[idx], device=DEVICE)
        item['labels2'] = torch.tensor(self.labels2[idx], device=DEVICE)
        item['labels3'] = torch.tensor(self.labels3[idx], device=DEVICE)
        item['labels4'] = torch.tensor(self.labels4[idx], device=DEVICE)
        return item



# Training and evaluation logic for separate models
def compute_accuracy(preds, labels):
    return accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())


# Split data
train_texts, val_texts, train_labels1, val_labels1, train_labels2, val_labels2, train_labels3, val_labels3 = train_test_split(
    df['description'], df['supergroup'], df['group'], df['module'], test_size=0.2, random_state=42
)

train_texts, val_texts, train_labels1, val_labels1, train_labels2, val_labels2, train_labels3, val_labels3, train_labels4, val_labels4 = train_test_split(
    df['description'], df['supergroup'], df['group'], df['module'], df['brand'], test_size=0.2, random_state=42
)

train_dataset = ProductDataset(train_texts.tolist(), train_labels1.tolist(), train_labels2.tolist(), train_labels3.tolist(), train_labels4.tolist())
val_dataset = ProductDataset(val_texts.tolist(), val_labels1.tolist(), val_labels2.tolist(), val_labels3.tolist(), val_labels4.tolist())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


# Initialize separate models
supergroup_model = torch.load('/kaggle/input/new-indoml-model/supergroup_model.pth').to(DEVICE)
group_model = torch.load('/kaggle/input/new-indoml-model/group_model.pth').to(DEVICE)
module_model = torch.load('/kaggle/input/new-indoml-model/module_model.pth').to(DEVICE)
brand_model = torch.load('/kaggle/input/new-indoml-model/brand_model.pth').to(DEVICE)

optimizer_supergroup = AdamP(supergroup_model.parameters(), lr=LEARNING_RATE)
optimizer_group = AdamP(group_model.parameters(), lr=LEARNING_RATE)
optimizer_module = AdamP(module_model.parameters(), lr=LEARNING_RATE)
optimizer_brand = AdamP(brand_model.parameters(), lr=LEARNING_RATE)

scaler = GradScaler()
scheduler_supergroup = ReduceLROnPlateau(optimizer_supergroup, mode='min', patience=PATIENCE_LR, factor=0.5, verbose=True)
scheduler_group = ReduceLROnPlateau(optimizer_group, mode='min', patience=PATIENCE_LR, factor=0.5, verbose=True)
scheduler_module = ReduceLROnPlateau(optimizer_module, mode='min', patience=PATIENCE_LR, factor=0.5, verbose=True)
scheduler_brand = ReduceLROnPlateau(optimizer_brand, mode='min', patience=PATIENCE_LR, factor=0.5, verbose=True)


def validate(supergroup_model, group_model, module_model, brand_model, val_loader):
    supergroup_model.eval()
    group_model.eval()
    module_model.eval()
    brand_model.eval()

    total_val_loss = 0
    total_val_acc1, total_val_acc2, total_val_acc3, total_val_acc4 = 0, 0, 0, 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels1, labels2, labels3, labels4 = batch['labels1'], batch['labels2'], batch['labels3'], batch['labels4']

            # Supergroup prediction
            logits1 = supergroup_model(input_ids=input_ids, attention_mask=attention_mask)
            loss1 = nn.CrossEntropyLoss()(logits1, labels1)
            supergroup_preds = torch.argmax(torch.softmax(logits1, dim=1), dim=1)

            # Group prediction
            logits2 = group_model(input_ids=input_ids, attention_mask=attention_mask)
            loss2 = nn.CrossEntropyLoss()(logits2, labels2)
            group_preds = group_model.predict(input_ids=input_ids, attention_mask=attention_mask, supergroup_preds=supergroup_preds)

            # Module prediction
            logits3 = module_model(input_ids=input_ids, attention_mask=attention_mask)
            loss3 = nn.CrossEntropyLoss()(logits3, labels3)
            module_preds = module_model.predict(input_ids=input_ids, attention_mask=attention_mask, group_preds=group_preds)

            # Brand prediction
            logits4 = brand_model(input_ids=input_ids, attention_mask=attention_mask)
            loss4 = nn.CrossEntropyLoss()(logits4, labels4)
            brand_preds = brand_model.predict(input_ids=input_ids, attention_mask=attention_mask, module_preds=module_preds)

            # Calculate total loss and accuracy for validation
            total_val_loss += (loss1.item() + loss2.item() + loss3.item() + loss4.item())
            total_val_acc1 += compute_accuracy(supergroup_preds, labels1)
            total_val_acc2 += compute_accuracy(group_preds, labels2)
            total_val_acc3 += compute_accuracy(module_preds, labels3)
            total_val_acc4 += compute_accuracy(brand_preds, labels4)

    avg_val_loss = total_val_loss / len(val_loader)
    avg_val_acc1 = total_val_acc1 / len(val_loader)
    avg_val_acc2 = total_val_acc2 / len(val_loader)
    avg_val_acc3 = total_val_acc3 / len(val_loader)
    avg_val_acc4 = total_val_acc4 / len(val_loader)

    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Supergroup Accuracy: {avg_val_acc1:.4f}")
    print(f"Group Accuracy: {avg_val_acc2:.4f}")
    print(f"Module Accuracy: {avg_val_acc3:.4f}")
    print(f"Brand Accuracy: {avg_val_acc4:.4f}")

    return avg_val_loss, avg_val_acc1, avg_val_acc2, avg_val_acc3, avg_val_acc4



# Training loop
# Training loop (now with brand model)
for epoch in range(NUM_EPOCHS):
    supergroup_model.train()
    group_model.train()
    module_model.train()
    brand_model.train()
    
    total_train_loss = 0
    total_train_acc1, total_train_acc2, total_train_acc3, total_train_acc4 = 0, 0, 0, 0

    with tqdm(total=len(train_loader), desc=f'Training Epoch {epoch+1}/{NUM_EPOCHS}', unit='batch', leave=False) as pbar:
        for batch in train_loader:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels1, labels2, labels3, labels4 = batch['labels1'], batch['labels2'], batch['labels3'], batch['labels4']

            # Supergroup model
            with torch.amp.autocast('cuda'):
                logits1 = supergroup_model(input_ids=input_ids, attention_mask=attention_mask)
                loss1 = nn.CrossEntropyLoss()(logits1, labels1)
            scaler.scale(loss1).backward()
            scaler.step(optimizer_supergroup)
            scaler.update()
            optimizer_supergroup.zero_grad()

            supergroup_preds = torch.argmax(torch.softmax(logits1, dim=1), dim=1)

            # Group model
            with torch.amp.autocast('cuda'):
                logits2 = group_model(input_ids=input_ids, attention_mask=attention_mask)
                loss2 = nn.CrossEntropyLoss()(logits2, labels2)
            scaler.scale(loss2).backward()
            scaler.step(optimizer_group)
            scaler.update()
            optimizer_group.zero_grad()

            group_preds = group_model.predict(input_ids=input_ids, attention_mask=attention_mask, supergroup_preds=supergroup_preds)

            # Module model
            with torch.amp.autocast('cuda'):
                logits3 = module_model(input_ids=input_ids, attention_mask=attention_mask)
                loss3 = nn.CrossEntropyLoss()(logits3, labels3)
            scaler.scale(loss3).backward()
            scaler.step(optimizer_module)
            scaler.update()
            optimizer_module.zero_grad()

            module_preds = module_model.predict(input_ids=input_ids, attention_mask=attention_mask, group_preds=group_preds)

            # Brand model
            with torch.amp.autocast('cuda'):
                logits4 = brand_model(input_ids=input_ids, attention_mask=attention_mask)
                loss4 = nn.CrossEntropyLoss()(logits4, labels4)
            scaler.scale(loss4).backward()
            scaler.step(optimizer_brand)
            scaler.update()
            optimizer_brand.zero_grad()

            brand_preds = brand_model.predict(input_ids=input_ids, attention_mask=attention_mask, module_preds=module_preds)

            total_train_loss += (loss1.item() + loss2.item() + loss3.item() + loss4.item())
            total_train_acc1 += compute_accuracy(supergroup_preds, labels1)
            total_train_acc2 += compute_accuracy(group_preds, labels2)
            total_train_acc3 += compute_accuracy(module_preds, labels3)
            total_train_acc4 += compute_accuracy(brand_preds, labels4)

            pbar.update(1)

    avg_train_loss = total_train_loss / len(train_loader)
    avg_train_acc1 = total_train_acc1 / len(train_loader)
    avg_train_acc2 = total_train_acc2 / len(train_loader)
    avg_train_acc3 = total_train_acc3 / len(train_loader)
    avg_train_acc4 = total_train_acc4 / len(train_loader)

    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Train Loss: {avg_train_loss:.4f}")
    print(f"Supergroup Accuracy: {avg_train_acc1:.4f}")
    print(f"Group Accuracy: {avg_train_acc2:.4f}")
    print(f"Module Accuracy: {avg_train_acc3:.4f}")
    print(f"Brand Accuracy: {avg_train_acc4:.4f}")

    # Validation step
    avg_val_loss, avg_val_acc1, avg_val_acc2, avg_val_acc3, avg_val_acc4 = validate(supergroup_model, group_model, module_model, brand_model, val_loader)
    
    scheduler_supergroup.step(avg_val_loss)
    scheduler_group.step(avg_val_loss)
    scheduler_module.step(avg_val_loss)
    scheduler_brand.step(avg_val_loss)
    print("*"*15)

tokenizer_config.json:   0%|          | 0.00/151 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/tmp/ipykernel_30/2373772054.py:176: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  supergroup_model = torch.load('/kaggle/input/new-indoml-model/supergroup_model.pth').to(DE

In [14]:
import time
from torch.profiler import profile, record_function, ProfilerActivity

In [21]:
supergroup_model.train()
sample_batch = next(iter(train_loader))  # Get one sample batch
sample_batch = {k: v.to(DEVICE) for k, v in sample_batch.items()}

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], with_flops=True) as prof:
    input_ids = sample_batch['input_ids'].to(DEVICE)
    attention_mask = sample_batch['attention_mask'].to(DEVICE)
    labels1, labels2, labels3, labels4 = sample_batch['labels1'], sample_batch['labels2'], sample_batch['labels3'], sample_batch['labels4']

       
    with torch.amp.autocast('cuda'):
        logits1 = supergroup_model(input_ids=input_ids, attention_mask=attention_mask)
        loss1 = nn.CrossEntropyLoss()(logits1, labels1)
    scaler.scale(loss1).backward()
    scaler.step(optimizer_supergroup)
    scaler.update()
    optimizer_supergroup.zero_grad()

    # Group model
    with torch.amp.autocast('cuda'):
        logits2 = group_model(input_ids=input_ids, attention_mask=attention_mask)
        loss2 = nn.CrossEntropyLoss()(logits2, labels2)
    scaler.scale(loss2).backward()
    scaler.step(optimizer_group)
    scaler.update()
    optimizer_group.zero_grad()

    with torch.amp.autocast('cuda'):
        logits3 = module_model(input_ids=input_ids, attention_mask=attention_mask)
        loss3 = nn.CrossEntropyLoss()(logits3, labels3)
    scaler.scale(loss3).backward()
    scaler.step(optimizer_module)
    scaler.update()
    optimizer_module.zero_grad()


    with torch.amp.autocast('cuda'):
        logits4 = brand_model(input_ids=input_ids, attention_mask=attention_mask)
        loss4 = nn.CrossEntropyLoss()(logits4, labels4)
    scaler.scale(loss4).backward()
    scaler.step(optimizer_brand)
    scaler.update()
    optimizer_brand.zero_grad()


print(prof.key_averages().table(sort_by="flops",row_limit=10))
print("GFLOPs during training") #GigaFLOPs
print(sum(k.flops for k in prof.key_averages())/1e9)

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  Total MFLOPs  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               aten::mm         0.93%       9.303ms         1.62%      16.119ms      54.457us      69.875ms        10.64%      69.875ms     236.063us           296    435493.405  
                                            aten::addmm         3.52%      35.038ms         9.43%      93.855ms     617.467us      38.610ms         5.8

In [25]:
def predict(supergroup_model, group_model, module_model, brand_model, tokenizer, text):
    start = time.time()
    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],with_flops=True) as prof:
        inputs = tokenizer(text, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors="pt")
        with torch.no_grad():
            input_ids = inputs['input_ids'].to(DEVICE)
            attention_mask = inputs['attention_mask'].to(DEVICE)

            # Supergroup prediction
            logits1 = supergroup_model(input_ids=input_ids, attention_mask=attention_mask)
            supergroup_preds = torch.argmax(torch.softmax(logits1, dim=1), dim=1)

            # Group prediction
            logits2 = group_model(input_ids=input_ids, attention_mask=attention_mask)
            group_preds = group_model.predict(input_ids=input_ids, attention_mask=attention_mask, supergroup_preds=supergroup_preds)

            # Module prediction
            logits3 = module_model(input_ids=input_ids, attention_mask=attention_mask)
            module_preds = module_model.predict(input_ids=input_ids, attention_mask=attention_mask, group_preds=group_preds)

            # Brand prediction
            logits4 = brand_model(input_ids=input_ids, attention_mask=attention_mask)
            brand_preds = brand_model.predict(input_ids=input_ids, attention_mask=attention_mask, module_preds=module_preds)

        supergroup = supergroup_encoder.inverse_transform([supergroup_preds.cpu().numpy()]),  # Detach from GPU, move to CPU, convert to NumPy, then inverse transform
        group = group_encoder.inverse_transform([group_preds.cpu().numpy()]),       # Similar for group
        module = module_encoder.inverse_transform([module_preds.cpu().numpy()]),      # Similar for module
        brand = brand_encoder.inverse_transform([brand_preds.cpu().numpy()])    
    print("Inference time :"+str(time.time()-start))
    #print(prof.key_averages().table(sort_by="flops",row_limit=10))
    print("GFLOPs during testing") #GigaFLOPs
    print(sum(k.flops for k in prof.key_averages())/1e9)
    return supergroup,group,module,brand
# Example prediction
sample_text = "14 in hybrid blade"
predictions = predict(supergroup_model, group_model, module_model, brand_model, tokenizer, sample_text)
print(predictions)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

Inference time :0.6291525363922119
GFLOPs during testing
11.92008192
((array(['home do it yourself'], dtype=object),), (array(['home do it yourself detail unknown total'], dtype=object),), (array(['home do it yourself'], dtype=object),), array(['receipt all'], dtype=object))


In [ ]:
test_df = pd.read_json('/kaggle/input/new-codalab-final-test/final_test_data/final_test_data.features',lines=True)

In [ ]:
test_df["description"] = test_df["description"] + " " +test_df["retailer"].astype(str) + " "+ test_df["price"].astype(str)
test_df["description"] = test_df["description"].astype(str)

In [ ]:
class TestDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        
        # Tokenizing the input text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            return_attention_mask=True,
            truncation=True,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
        }

    
def create_test_loader(test_texts, tokenizer, max_length, batch_size):
    test_dataset = TestDataset(
        texts=test_texts,
        tokenizer=tokenizer,
        max_length=max_length
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return test_loader


tokenizer = AutoTokenizer.from_pretrained("canwenxu/BERT-of-Theseus-MNLI")

In [ ]:
test_loader = create_test_loader(test_df["description"].tolist(),tokenizer,20,32)

In [ ]:
def test(supergroup_model, group_model, module_model, brand_model, test_loader):
    supergroup_model.eval()
    group_model.eval()
    module_model.eval()
    brand_model.eval()

    supergroup_preds_list = []
    group_preds_list = []
    module_preds_list = []
    brand_preds_list = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)

            # 1. Predict supergroup
            logits1 = supergroup_model(input_ids=input_ids, attention_mask=attention_mask)
            supergroup_preds = torch.argmax(torch.softmax(logits1, dim=1), dim=1)

            # 2. Predict group using predicted supergroup
            group_preds = group_model.predict(input_ids=input_ids, attention_mask=attention_mask, supergroup_preds=supergroup_preds)

            # 3. Predict module using predicted group
            module_preds = module_model.predict(input_ids=input_ids, attention_mask=attention_mask, group_preds=group_preds)

            # 4. Predict brand using predicted module
            brand_preds = brand_model.predict(input_ids=input_ids, attention_mask=attention_mask, module_preds=module_preds)

            # Store predictions for final evaluation or submission
            supergroup_preds_list.extend(supergroup_preds.cpu().numpy())
            group_preds_list.extend(group_preds.cpu().numpy())
            module_preds_list.extend(module_preds.cpu().numpy())
            brand_preds_list.extend(brand_preds.cpu().numpy())

    # Return all predictions for further analysis or submission
    return supergroup_preds_list, group_preds_list, module_preds_list, brand_preds_list


# Making predictions with all models
supergroup_pred, group_pred, module_pred, brand_pred = test(supergroup_model, group_model, module_model, brand_model, test_loader)

print("Predictions are complete, now detaching and inverse transforming the predictions.")

# Detach, move to CPU, and inverse transform the predictions
inverse_predictions = [
    (
        supergroup_encoder.inverse_transform([supergroup_pred[i]]),  # Detach from GPU, move to CPU, convert to NumPy, then inverse transform
        group_encoder.inverse_transform([group_pred[i]]),       # Similar for group
        module_encoder.inverse_transform([module_pred[i]]),      # Similar for module
        brand_encoder.inverse_transform([brand_pred[i]])         # Similar for brand
    )
    for i in range(len(group_pred))  # Iterate through each set of predictions
]

# Extract predictions into separate lists
supergroup = []
group = []
module = []
brand = []

for (sg, g, md, br) in inverse_predictions:
    supergroup.append(sg[0])
    group.append(g[0])
    module.append(md[0])
    brand.append(br[0])

# Adding the predictions to the test DataFrame
test_df['supergroup'] = supergroup
test_df['group'] = group
test_df['module'] = module
test_df['brand'] = brand

In [ ]:
test_df

In [ ]:
test_df.to_csv('/kaggle/working/predictions.csv')

In [ ]:
torch.save(supergroup_model, 'supergroup_model.pth')
torch.save(group_model,'group_model.pth')
torch.save(module_model,'module_model.pth')
torch.save(brand_model,'brand_model.pth')